In [1]:
device = "cpu"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd drive/MyDrive/Project-NLP-2023

/content/drive/MyDrive/Project-NLP-2023


In [4]:
!pip install transformers
!pip install icecream

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from datasets import  Tokenized_Dataset

In [6]:
dataset = Tokenized_Dataset(json_file='negacio_uab_revised_version.json', tokenizer_name='bert-base-multilingual-cased')

do train_test split:

In [7]:
import torch
# Define the indices for train and test subsets
train_indices = range(0,len(dataset)-int(len(dataset)*0.3))
test_indices = range(len(dataset)-int(len(dataset)*0.3), len(dataset))

# Create Subset datasets based on the defined indices
train_data = torch.utils.data.Subset(dataset, train_indices)
val_data= torch.utils.data.Subset(dataset, test_indices)

In [8]:
from torch.utils.data import DataLoader
batch_size = 5
train_loader = DataLoader(train_data,batch_size)
val_loader = DataLoader(val_data,1)

Import pre-trained bert multillingual

In [9]:
from transformers import BertModel
bert = BertModel.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
import torch

Test a sample:

In [11]:
#sample = next(iter(train_loader))
#sample

In [12]:
#predictions = bert(sample["x"])
#predictions = predictions["last_hidden_state"]

In [13]:
#predictions.shape

In [14]:
#tags = sample["y"]
#tags.shape

In [15]:
#tags.shape

We'll define a tagger model that has a linear layer that helps project the last hidden state into the vocab we have. We'll further have a dropout for regularisation.

In [16]:
import torch.nn as nn

class BERT_Tagger(nn.Module):
    def __init__(self,
                 bert,
                 output_dim, 
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.fc = nn.Linear(embedding_dim,output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, tokens):
        
        bert_out = self.bert(tokens)["last_hidden_state"]
        
        predictions = self.fc(bert_out)
        
        return predictions

In [17]:
#bert_tagger = BERT_Tagger(bert,5,0.1)
#torch.save(bert_tagger.state_dict(),"finetuned_bert.pt")
bert_tagger = torch.load("finetuned_bert.pt",map_location=torch.device('cpu'))

In [18]:
torch.save

<function torch.serialization.save(obj: object, f: Union[str, os.PathLike, BinaryIO, IO[bytes]], pickle_module: Any = <module 'pickle' from '/usr/lib/python3.10/pickle.py'>, pickle_protocol: int = 2, _use_new_zipfile_serialization: bool = True) -> None>

In [19]:
#out = bert_tagger(sample["x"])
#out.shape

In [20]:
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]]).to(device)

In [21]:
min_loss = 100000
def train(model, epochs, dataloader, optimizer_full, criterion_specific, criterion, tag_pad_idx):    
    model.train()

    for i in range(epochs):
        for j, batch in enumerate(dataloader):
            tokens = batch["x"].to(device)
            tags = batch["y"].to(device)
            #look if all tags in the batch are none, if so skip
            #if torch.equal(tags, torch.tensor([[dataset.uniq_tags.index("NONE")]*tags.shape[1]]).to(device)) :
            #    continue #skip batch
                     
            optimizer.zero_grad()
            
            #text = [sent len, batch size]
            
            predictions = model(tokens)
            predictions = predictions.view(-1, predictions.shape[-1]) #merge sent len and batch dimensions

            tags = tags.view(-1)
            #predictions  = [sent len * batch size, output dim]
            #tags = [sent len * batch size]
            
            loss_full = criterion_full(predictions, tags)
            loss_specific = criterion_specific(predictions, tags)
            loss = loss_full*0.2 + loss_specific*0.8
                    
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)
            
            if loss.item() < min_loss:
              torch.save(model,"finetuned_bert.pt")

            loss.backward()
            
            optimizer.step()


            print(f"epoch:{i} batch:{j} loss:{loss.item()} acc:{acc.item()}")

In [22]:
#make the criterion not count loss on "NONE" tag
criterion_specific = nn.CrossEntropyLoss(ignore_index = dataset.uniq_tags.index("NONE"))
criterion_full = nn.CrossEntropyLoss()

In [23]:
import torch.optim as optim
LEARNING_RATE = 5e-5 #as recomended in BERT paper
optimizer = optim.Adam(bert_tagger.parameters(), lr = LEARNING_RATE)

AttributeError: ignored

In [ ]:
bert_tagger = bert_tagger.to(device)

In [ ]:
tag_pad_idx = dataset.uniq_tags.index("NONE")

In [ ]:
train(bert_tagger,1,train_loader,optimizer,criterion_full,criterion_specific,10)

In [ ]:
train(bert_tagger,2,train_loader,optimizer,criterion,10)

Test the model qualititavely:

In [ ]:
from colorama import Fore, Back, Style

In [ ]:
def eval(model,dataset,n_batches,tag_pad_idx,uniq_tags):
    colors = [Fore.BLUE,Fore.RED,Fore.GREEN,Fore.CYAN,Fore.WHITE]
    reference_tag_txt = [colors[i]+uniq_tags[i] for i in range(len(uniq_tags))]
    reference_tag_txt = " ".join(reference_tag_txt)
    
    for i in range(0,n_batches*512,512):
        batch = dataset.__getitem__(i)
        tokens = batch["x"].to(device).unsqueeze(0)
        tags = batch["y"].to(device).unsqueeze(0)
        tokens_txt = batch["x_ref"]
        predictions = model(tokens)
        predictions = predictions.view(-1, predictions.shape[-1]) #merge sent len and batch dimensions
        predictions = torch.argmax(predictions,axis=1).to("cpu").numpy()
        tags = tags.view(-1)
        
        txt = ""
        txt_pred = ""
        for tok,tag,pred in zip(tokens_txt,tags,predictions):
            if tok[0] == "#":
                txt += colors[tag]+tok.replace("#","")
                txt_pred += colors[pred]+tok.replace("#","")

            else:
                txt += " " + colors[tag]+tok.replace("#","")
                txt_pred += " " + colors[pred]+tok.replace("#","")

        print(reference_tag_txt)
        print(Fore.WHITE+"-------------------True-------------------")
        print(txt)
        print(Fore.WHITE+"-------------------True-------------------")
        print(Fore.WHITE+"-------------------Pred-------------------")
        print(txt_pred)
        print(Fore.WHITE+"-------------------Pred-------------------")
        

In [ ]:
eval(bert_tagger,val_data,10,tag_pad_idx,dataset.uniq_tags)